In [222]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

In [223]:
test_pred = pd.read_csv("test_pred_tensor_flow.csv",encoding = "ISO-8859-1")

In [224]:
train_df=pd.read_csv('split_train_pred_tensor_flow.csv')
test_df=pd.read_csv('test_limpio_sin_vect_textos.csv')
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")

In [225]:
test_df['pred_tf'] = test_pred.tensor_flow_pred

In [226]:
#Agrego las otras predicciones como features
pred_test_catboost=pd.read_csv('feature_pred/prediccion_test_catBoost.csv')
pred_test_bayes=pd.read_csv('feature_pred/prediccion_test_bayes.csv')
pred_test_svm=pd.read_csv('feature_pred/prediccion_test_SVM.csv')

In [227]:
test_df['pred_catboost'] = pred_test_catboost.target
test_df['pred_bayes'] = pred_test_bayes.target
test_df['pred_svm'] = pred_test_svm.target

In [228]:
pred_tf = pd.read_csv("test_pred_tensor_flow.csv",encoding = "ISO-8859-1")

In [229]:
sample.target = pred_tf.tensor_flow_pred

In [230]:
def redond(x):
    result=0
    if(x>0.5):
        result=1
    return(result)

In [231]:
sample.target = sample.target.apply(lambda x: redond(x))

In [232]:
sample.target.value_counts()

0    2042
1    1221
Name: target, dtype: int64

In [233]:
sample.to_csv('pred_luke.csv', index=False)

# Veo la distribucion de valores mal predichos

In [234]:
train_df['tensor_flow_pred_redond'] = train_df['tensor_flow_pred'].apply(lambda x: redond(x))

In [235]:
def redond_decim(x):
    result=1
    if(x<0.1):
        result=0.1
    elif(x<0.2):
        result=0.2
    elif(x<0.3):
        result=0.3
    elif(x<0.4):
        result=0.4
    elif(x<0.5):
        result=0.5
    elif(x<0.6):
        result=0.6
    elif(x<0.7):
        result=0.7
    elif(x<0.8):
        result=0.8
    elif(x<0.9):
        result=0.9
    return(result)

In [236]:
train_df['tensor_flow_pred_redond_decim'] = train_df['tensor_flow_pred'].apply(lambda x: redond_decim(x))

# Cuento la cantidad de elementos predichos correctamente por TF segun su porcentaje

In [237]:
valores_mal_predichos_tensor = train_df[train_df['tensor_flow_pred_redond']==train_df.target]
valores_mal_predichos_tensor.shape

(1237, 2844)

In [238]:
dis_tf = valores_mal_predichos_tensor.groupby('tensor_flow_pred_redond_decim').agg({'tensor_flow_pred_redond':'count'})
dis_tf

,tensor_flow_pred_redond
tensor_flow_pred_redond_decim,
0.1,239
0.2,234
0.3,127
0.4,67
0.5,28
0.6,40
0.7,38
0.8,33
0.9,61


# Cuento la cantidad de elementos predichos correctamente por CatBoost segun su porcentaje

In [239]:
valores_mal_predichos_catboost = train_df[train_df['pred_catbooost']==train_df.target]
valores_mal_predichos_catboost.shape

(1169, 2844)

In [240]:
dis_cb = valores_mal_predichos_catboost.groupby('tensor_flow_pred_redond_decim').agg({'pred_catbooost':'count'})
dis_cb

,pred_catbooost
tensor_flow_pred_redond_decim,
0.1,231
0.2,225
0.3,122
0.4,62
0.5,27
0.6,47
0.7,32
0.8,31
0.9,52


# Cuento la cantidad de elementos predichos correctamente por bayes segun su porcentaje

In [241]:
valores_mal_predichos_bayes = train_df[train_df['pred_bayes']==train_df.target]
valores_mal_predichos_bayes.shape

(1205, 2844)

In [242]:
dis_bay = valores_mal_predichos_bayes.groupby('tensor_flow_pred_redond_decim').agg({'pred_bayes':'count'})
dis_bay

,pred_bayes
tensor_flow_pred_redond_decim,
0.1,237
0.2,230
0.3,126
0.4,60
0.5,33
0.6,48
0.7,33
0.8,33
0.9,53


# Cuento la cantidad de elementos predichos correctamente por SVM segun su porcentaje

In [243]:
valores_mal_predichos_svm = train_df[train_df['pred_svm']==train_df.target]
valores_mal_predichos_svm.shape

(1202, 2844)

In [244]:
dis_svm = valores_mal_predichos_bayes.groupby('tensor_flow_pred_redond_decim').agg({'pred_svm':'count'})
dis_svm

,pred_svm
tensor_flow_pred_redond_decim,
0.1,237
0.2,230
0.3,126
0.4,60
0.5,33
0.6,48
0.7,33
0.8,33
0.9,53


# Cuento la cantidad de elementos predichos correctamente por el promedio de las tf,catboost, y svm segun su porcentaje

In [245]:
train_df['ens_tf_cb_svm'] = (train_df['tensor_flow_pred_redond']+train_df['pred_catbooost']+train_df['pred_svm'])/3
train_df['ens_tf_cb_svm'] = train_df['ens_tf_cb_svm'].apply(lambda x: redond(x))

In [246]:
valores_mal_predichos_ens = train_df[train_df['ens_tf_cb_svm']==train_df.target]
valores_mal_predichos_ens.shape

(1239, 2845)

In [247]:
dis_ens_tf_cb_svm = valores_mal_predichos_ens.groupby('tensor_flow_pred_redond_decim').agg({'ens_tf_cb_svm':'count'})
dis_ens_tf_cb_svm

,ens_tf_cb_svm
tensor_flow_pred_redond_decim,
0.1,238
0.2,234
0.3,130
0.4,67
0.5,31
0.6,49
0.7,33
0.8,35
0.9,54


# Cuento la cantidad de elementos predichos correctamente por el promedio de las tf,bayes, y svm segun su porcentaje

In [248]:
train_df['ens_tf_bayes_svm'] = (train_df['tensor_flow_pred_redond']+train_df['pred_bayes']+train_df['pred_svm'])/3
train_df['ens_tf_bayes_svm'] = train_df['ens_tf_bayes_svm'].apply(lambda x: redond(x))

In [249]:
valores_mal_predichos_ens = train_df[train_df['ens_tf_bayes_svm']==train_df.target]
valores_mal_predichos_ens.shape

(1228, 2846)

In [250]:
dis_ens_tf_bayes_svm = valores_mal_predichos_ens.groupby('tensor_flow_pred_redond_decim').agg({'ens_tf_bayes_svm':'count'})
dis_ens_tf_bayes_svm

,ens_tf_bayes_svm
tensor_flow_pred_redond_decim,
0.1,238
0.2,232
0.3,129
0.4,62
0.5,34
0.6,47
0.7,32
0.8,32
0.9,55


# Cuento la cantidad de elementos predichos correctamente por el promedio de las tf,cb, y bayes segun su porcentaje

In [251]:
train_df['ens_tf_bayes_cb'] = (train_df['tensor_flow_pred_redond']+train_df['pred_bayes']+train_df['pred_catbooost'])/3
train_df['ens_tf_bayes_cb'] = train_df['ens_tf_bayes_cb'].apply(lambda x: redond(x))

In [252]:
valores_mal_predichos_ens = train_df[train_df['ens_tf_bayes_cb']==train_df.target]
valores_mal_predichos_ens.shape

(1237, 2847)

In [253]:
dis_ens_tf_bayes_cb = valores_mal_predichos_ens.groupby('tensor_flow_pred_redond_decim').agg({'ens_tf_bayes_cb':'count'})
dis_ens_tf_bayes_cb

,ens_tf_bayes_cb
tensor_flow_pred_redond_decim,
0.1,238
0.2,234
0.3,130
0.4,68
0.5,29
0.6,48
0.7,34
0.8,32
0.9,58


# Cuento la cantidad de elementos predichos correctamente por el promedio de las svm,cb, y bayes segun su porcentaje

In [254]:
train_df['ens_svm_bayes_cb'] = (train_df['pred_svm']+train_df['pred_bayes']+train_df['pred_catbooost'])/3
train_df['ens_svm_bayes_cb'] = train_df['ens_svm_bayes_cb'].apply(lambda x: redond(x))

In [255]:
valores_mal_predichos_ens = train_df[train_df['ens_svm_bayes_cb']==train_df.target]
valores_mal_predichos_ens.shape

(1218, 2848)

In [256]:
dis_ens_svm_bayes_cb = valores_mal_predichos_ens.groupby('tensor_flow_pred_redond_decim').agg({'ens_svm_bayes_cb':'count'})
dis_ens_svm_bayes_cb

,ens_svm_bayes_cb
tensor_flow_pred_redond_decim,
0.1,238
0.2,232
0.3,129
0.4,63
0.5,36
0.6,48
0.7,31
0.8,37
0.9,51


# Junto todos

In [257]:
dis_tf['pred_cb'] = dis_cb['pred_catbooost']
dis_tf['pred_bayes'] = dis_bay['pred_bayes']
dis_tf['pred_svm'] = dis_svm['pred_svm']
dis_tf['pred_ens_tf_cb_svm'] = dis_ens_tf_cb_svm['ens_tf_cb_svm']
dis_tf['ens_tf_bayes_svm'] = dis_ens_tf_bayes_svm['ens_tf_bayes_svm']
dis_tf['ens_tf_bayes_cb'] = dis_ens_tf_bayes_cb['ens_tf_bayes_cb']
dis_tf['ens_svm_bayes_cb'] = dis_ens_svm_bayes_cb['ens_svm_bayes_cb']
dis_tf

,tensor_flow_pred_redond,pred_cb,pred_bayes,pred_svm,pred_ens_tf_cb_svm,ens_tf_bayes_svm,ens_tf_bayes_cb,ens_svm_bayes_cb
tensor_flow_pred_redond_decim,,,,,,,,
0.1,239,231,237,237,238,238,238,238
0.2,234,225,230,230,234,232,234,232
0.3,127,122,126,126,130,129,130,129
0.4,67,62,60,60,67,62,68,63
0.5,28,27,33,33,31,34,29,36
0.6,40,47,48,48,49,47,48,48
0.7,38,32,33,33,33,32,34,31
0.8,33,31,33,33,35,32,32,37
0.9,61,52,53,53,54,55,58,51


# La moraleja de esto: En los valores cuya prediccion de tensor flow esta entre 0.4<x<=0.6, me conviene hacer un ensamble, lo pruebo con test y lo subo

In [258]:
test_df['ens'] = (test_df['pred_tf']+test_df['pred_catboost']+test_df['pred_svm'])/3

In [259]:
test_df['target'] = np.where(((test_df['pred_tf']>0.4) & (test_df['pred_tf']<=0.6)), test_df['ens'], test_df['pred_tf'])

In [260]:
test_df['target'] = test_df['target'].apply(lambda x: redond(x))

In [261]:
sample.target = test_df.target

In [262]:
sample.target.value_counts()

0    2094
1    1169
Name: target, dtype: int64

In [263]:
sample.to_csv('neural_network_with_ens.csv', index=False)